In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import transformers
import trl
import pandas as pd
import datasets
import torch

[2024-01-08 20:11:12,764] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
model_id = "google/flan-t5-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [4]:
prompt = """
What is the full lyrics of the song Sound of Silence by Simon & Garfunkel?
""".strip()

pipeline = transformers.pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=model.device)
pipeline(prompt, max_new_tokens=100)

[{'generated_text': '"Son of Silence" is a song by Simon & Garfunkel, released in April 1964.'}]

In [5]:
dummy_dataset = [
    {
        "prompt": prompt,
        "rejected": "\"Son of Silence\" is a song by Simon & Garfunkel, released in April 1964.'",
        "chosen": "foo bar buzz.",
    },
    {
        "prompt": prompt,
        "rejected": "foo bar buzz.",
        "chosen": "lorem ipsum dolor sit amet.",
    },
    {
        "prompt": prompt,
        "rejected": "lorem ipsum dolor sit amet.",
        "chosen": "This is a simple test if DPO training will transitively converge to the highest ranked answer.",
    },
    {
        "prompt": prompt,
        "rejected": "This is a simple test if DPO training will transitively converge to the highest ranked answer.",
        "chosen": "The answer to the ultimate question of life, the universe, and everything is 42.",
    },
]

dummy_dataset = datasets.Dataset.from_list(dummy_dataset)
dummy_dataset

Dataset({
    features: ['prompt', 'rejected', 'chosen'],
    num_rows: 4
})

In [6]:
training_args = transformers.TrainingArguments(
    num_train_epochs=100,
    do_train=True,
    do_eval=False,
    per_device_train_batch_size=4,
    output_dir="./output",
    report_to="none",
)

In [7]:
trainer = trl.DPOTrainer(
    model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dummy_dataset,
)

/var/tmp/xkadlci2/.conda/envs/calc-x/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:263: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the DPOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/var/tmp/xkadlci2/.conda/envs/calc-x/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:270: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the DPOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/var/tmp/xkadlci2/.conda/envs/calc-x/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:278: UserWarning: When using DPODataCollatorWithPadding with an encoder decoder architecture, you should set `max_target_length` in the DPOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/var/tmp/xkadlci2/.conda/envs/calc-x/lib/pytho

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [8]:
trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss


TrainOutput(global_step=100, training_loss=0.035196692943573, metrics={'train_runtime': 26.7272, 'train_samples_per_second': 14.966, 'train_steps_per_second': 3.742, 'total_flos': 0.0, 'train_loss': 0.035196692943573, 'epoch': 100.0})

In [9]:
pipeline = transformers.pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=model.device)

In [11]:
pipeline(prompt, max_new_tokens=40)

[{'generated_text': 'The answer to the ultimate question of life, the universe, and everything is 42.'}]